<a href="https://colab.research.google.com/github/dhanmoti/ml-exercise/blob/smu-fnce685-project/SMU_FNCE685_G2_PD_Prediction_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Define the Problem & **Objectives**
## Objective
* **Static PD Model**: Predicts probability of default (PD) at loan origination based on borrower characteristics at application.

* **Dynamic PD Model**: Adjusts PD over time by incorporating loan performance trends and changing borrower behavior.

### Hypothesis
A dynamic PD model provides better predictions by adapting to changing borrower conditions, whereas a static model can only be used at loan origination stage.

# Install & Import

In [3]:
# Install packages if required
#!pip install -q kaggle
#!pip install xgboost catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.2 MB/s eta 0:00:00


In [4]:
# Import
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from catboost import CatBoostClassifier
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix, roc_curve, auc
import warnings
warnings.filterwarnings('ignore')


In [5]:
 # Setting up kaggle
# from google.colab import files

# files.upload()
# ! mkdir ~/.kaggle

#! cp kaggle.json ~/.kaggle/

Saving kaggle.json to kaggle.json


In [6]:
#! chmod 600 ~/.kaggle/kaggle.json

In [8]:
# Download Kaggle data
#! kaggle competitions download -c loan-default-pred

 37% 3.00M/8.12M [00:00<00:00, 30.6MB/s]
100% 8.12M/8.12M [00:00<00:00, 42.5MB/s]


In [9]:
#!unzip loan-default-pred.zip

Archive:  loan-default-pred.zip
  inflating: loan_sample_submission.csv  
  inflating: loan_test.csv           
  inflating: loan_train.csv          


# Reusable Functions

In [ ]:
def plot_correlation_heatmap(df, annot=False, figsize=(12, 8), cmap="coolwarm", linewidths=0.5, columns_subset=None):
    """
    Plots a heatmap of the correlation matrix for the given DataFrame.

    Parameters:
        df (pandas.DataFrame): The input DataFrame.
        annot (bool): Whether to display correlation coefficients.
                      Disabling it (annot=False) can speed up rendering.
        figsize (tuple): Size of the figure.
        cmap (str): Colormap.
        linewidths (float): Width of the lines that will divide each cell.
        columns_subset (list or None): If provided, only these columns will be used to compute the correlation.

    Returns:
        None: Displays the heatmap.
    """
    # If a subset of columns is provided, use it; otherwise, select numeric columns.
    if columns_subset is not None:
        data = df[columns_subset]
    else:
        data = df.select_dtypes(include=[np.number])

    # Compute the correlation matrix.
    corr_matrix = data.corr()

    # Create the heatmap.
    plt.figure(figsize=figsize)
    sns.heatmap(corr_matrix, annot=annot, fmt=".2f", cmap=cmap, linewidths=linewidths)
    plt.title("Feature Correlation Heatmap")
    plt.show()

In [ ]:
import numpy as np
import pandas as pd

def drop_highly_correlated_features(df, threshold=0.75):
    """
    Drops features from the DataFrame that are highly correlated with others using a vectorized approach.

    Parameters:
        df (pandas.DataFrame): The input DataFrame.
        threshold (float): The correlation threshold above which a feature is dropped.
                           (Default is 0.75)

    Returns:
        new_df (pandas.DataFrame): A new DataFrame with highly correlated features removed.
        dropped_features (list): A list of the feature names that were dropped.
    """
    # Compute the correlation matrix and take the absolute value
    corr_matrix = df.corr().abs()

    # Create a mask for the upper triangle (excluding the diagonal)
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Identify columns to drop: any column with a correlation value greater than the threshold
    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > threshold)]

    # Drop these columns from the DataFrame
    new_df = df.drop(columns=to_drop)

    return new_df, to_drop

In [ ]:
def preprocess_dataframe(df):
    """
    Preprocess the DataFrame by performing the following:
      1. Drop unnecessary columns.
      2. Convert interest rate ('int_rate') from a percentage string to float.
      3. Convert date columns ('issue_d' and 'last_pymnt_d') to datetime.
      4. Label encode specified ordinal columns.
      5. One-hot encode any remaining categorical columns.

    Parameters:
      df (pandas.DataFrame): The input DataFrame.

    Returns:
      df_encoded (pandas.DataFrame): The preprocessed DataFrame.
      label_encoders (dict): Dictionary of fitted label encoders for ordinal columns.
    """
    # Work on a copy to avoid modifying the original DataFrame.
    df_processed = df.copy()

    # 1. Drop unnecessary columns (ignore if any are missing).
    cols_to_drop = ['id', 'member_id', 'url', 'desc', 'title', 'emp_title']
    df_processed.drop(columns=cols_to_drop, errors='ignore', inplace=True)

    # 2. Convert interest rate from percentage string to float.
    if 'int_rate' in df_processed.columns:
        df_processed['int_rate'] = df_processed['int_rate'].str.rstrip('%').astype(float)

    # 3. Convert date columns to datetime objects.
    date_columns = {
        'issue_d': '%b-%y',
        'last_pymnt_d': '%b-%y'
    }
    for col, date_format in date_columns.items():
        if col in df_processed.columns:
            df_processed[col] = pd.to_datetime(df_processed[col], format=date_format)

    # 3.1 Convert date to ordinal numerical data
    df_processed = convert_datetime_columns(df_processed, method='ordinal')


    # 4. Label encode specified ordinal columns.
    ordinal_columns = ['term', 'grade', 'sub_grade', 'emp_length']
    label_encoders = {}
    for col in ordinal_columns:
        if col in df_processed.columns:
            le = LabelEncoder()
            df_processed[col] = le.fit_transform(df_processed[col])
            label_encoders[col] = le

    # 5. Identify any remaining categorical columns and apply one-hot encoding.
    categorical_cols = df_processed.select_dtypes(include=['object']).columns
    print("Categorical Columns for one-hot encoding:", list(categorical_cols))
    df_encoded = pd.get_dummies(df_processed, columns=categorical_cols, drop_first=True)

    # Optional: Check conversion of date columns if they exist.
    if 'issue_d' in df_encoded.columns and 'last_pymnt_d' in df_encoded.columns:
        print("Date conversion check:")
        print(df_encoded[['issue_d', 'last_pymnt_d']].head())

    return df_encoded, label_encoders

# Example usage:
# df = pd.read_csv('your_dataset.csv')
# df_preprocessed, encoders = preprocess_dataframe(df)
# df_preprocessed.head()


In [ ]:
import pandas as pd

def handle_missing_values(df, method='fill', numeric_strategy='median', categorical_strategy='mode'):
    """
    Handles missing values in the DataFrame.

    Parameters:
        df (pandas.DataFrame): The input DataFrame.
        method (str): Method to handle missing values. Options:
                      'fill' (default) to fill missing values,
                      'drop' to drop rows with missing values.
        numeric_strategy (str): Strategy for numeric columns when method='fill'.
                                Options: 'median' (default) or 'mean'.
        categorical_strategy (str): Strategy for categorical columns when method='fill'.
                                    Options: 'mode' (default). If another value is provided,
                                    missing values will be filled with the string "Missing".

    Returns:
        pandas.DataFrame: A new DataFrame with missing values handled.
    """
    df_handled = df.copy()

    if method == 'fill':
        # Fill numeric columns.
        numeric_cols = df_handled.select_dtypes(include=['number']).columns
        for col in numeric_cols:
            if numeric_strategy == 'median':
                df_handled[col].fillna(df_handled[col].median(), inplace=True)
            elif numeric_strategy == 'mean':
                df_handled[col].fillna(df_handled[col].mean(), inplace=True)
            else:
                raise ValueError("Invalid numeric_strategy. Use 'median' or 'mean'.")

        # Fill categorical columns.
        categorical_cols = df_handled.select_dtypes(include=['object']).columns
        for col in categorical_cols:
            if categorical_strategy == 'mode':
                # Use the most frequent value.
                df_handled[col].fillna(df_handled[col].mode()[0], inplace=True)
            else:
                # Fill with a custom string if desired.
                df_handled[col].fillna("Missing", inplace=True)

    elif method == 'drop':
        df_handled.dropna(inplace=True)
    else:
        raise ValueError("Invalid method. Use 'fill' or 'drop'.")

    return df_handled

# Example usage:
# df = pd.read_csv('your_dataset.csv')
# df_clean = handle_missing_values(df, method='fill', numeric_strategy='median', categorical_strategy='mode')
# df_clean.info()


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error

def run_lasso_feature_selection(df, target, test_size=0.2, random_state=42):
    """
    Splits the dataset, applies LassoCV for feature selection and regression,
    prints and returns the selected features and test set performance.

    Parameters:
        df (pandas.DataFrame): Input DataFrame with features and target.
        target (str): Name of the target column.
        test_size (float): Fraction of data to use as test set (default is 0.2).
        random_state (int): Random seed for reproducibility (default is 42).

    Returns:
        lasso_cv (LassoCV): The fitted LassoCV model.
        selected_features (list): List of feature names with non-zero coefficients.
        mse (float): Mean Squared Error on the test set.
    """
    # Separate features and target
    X = df.drop(columns=[target])
    y = df[target]

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        test_size=test_size,
                                                        random_state=random_state)

    # Apply Lasso with cross-validation to find the optimal regularization parameter
    lasso_cv = LassoCV(cv=5, random_state=random_state)
    lasso_cv.fit(X_train, y_train)

    # Identify features with non-zero coefficients
    selected_features = X.columns[lasso_cv.coef_ != 0]
    print("Selected features:", list(selected_features))

    # Evaluate performance on the test set
    y_pred = lasso_cv.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Squared Error:", mse)

    return lasso_cv, list(selected_features), mse

# Example usage:
# df = pd.read_csv('your_dataset.csv')
# model, features, error = run_lasso_feature_selection(df, target='target')


In [ ]:
import pandas as pd

def convert_datetime_columns(df, method='ordinal'):
    """
    Convert all datetime columns in a DataFrame to numeric values.

    Parameters:
        df (pandas.DataFrame): The input DataFrame.
        method (str): Conversion method. Options:
                      'ordinal' - converts to ordinal (days count) using toordinal().
                      'unix' - converts to Unix timestamp (seconds since epoch).

    Returns:
        df (pandas.DataFrame): The DataFrame with datetime columns converted to numeric values.
    """
    datetime_cols = df.select_dtypes(include=['datetime64[ns]']).columns
    for col in datetime_cols:
        if method == 'ordinal':
            # Convert each datetime to its ordinal if not null; otherwise, return None
            df[col] = df[col].apply(lambda x: x.toordinal() if pd.notnull(x) else None)
        elif method == 'unix':
            # Convert each datetime to a Unix timestamp if not null; otherwise, return None
            df[col] = df[col].apply(lambda x: int(x.timestamp()) if pd.notnull(x) else None)
        else:
            raise ValueError("Invalid method. Choose 'ordinal' or 'unix'.")
    return df

# Example usage:
# Assuming X_train and X_test are your training and testing DataFrames containing datetime columns:
# X_train = convert_datetime_columns(X_train, method='ordinal')
# X_test = convert_datetime_columns(X_test, method='ordinal')


In [ ]:
def standardize_numerical_features(X):
  # Scale numerical features [split first. do it on train set]
  scaler = StandardScaler()
  numeric_columns = X.select_dtypes(include=['int64', 'float64']).columns
  X[numeric_columns] = scaler.fit_transform(X[numeric_columns])
  return X

# Load

In [ ]:
# Load dataset
df_train = pd.read_csv("/content/loan_train.csv")

## Check Data Types & Convert Columns if Needed


In [138]:
df_preprocessed_train, encoders = preprocess_dataframe(df_train)
df_preprocessed_train.head()



Categorical Columns for one-hot encoding: ['home_ownership', 'verification_status', 'purpose', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'last_credit_pull_d']
Date conversion check:
     issue_d last_pymnt_d
0 2010-07-01   2012-09-01
1 2010-09-01   2013-05-01
2 2009-11-01   2011-05-01
3 2011-04-01   2013-07-01
4 2011-12-01   2013-09-01


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,...,last_credit_pull_d_Oct-15,last_credit_pull_d_Sep-07,last_credit_pull_d_Sep-08,last_credit_pull_d_Sep-09,last_credit_pull_d_Sep-10,last_credit_pull_d_Sep-11,last_credit_pull_d_Sep-12,last_credit_pull_d_Sep-13,last_credit_pull_d_Sep-14,last_credit_pull_d_Sep-15
0,10000,10000,9975.0,0,13.23,338.05,2,10,4,37000.0,...,False,False,False,False,False,False,False,False,False,False
1,14000,14000,13775.0,0,7.88,437.94,0,4,10,105000.0,...,False,False,False,False,False,False,False,False,False,False
2,9475,9475,9475.0,0,8.94,301.04,0,4,9,27000.0,...,False,False,False,False,False,False,False,False,False,False
3,4000,4000,4000.0,0,7.29,124.04,0,3,1,35000.0,...,False,False,False,False,False,False,False,False,False,False
4,3600,3600,3600.0,0,6.62,110.54,0,1,0,60000.0,...,False,False,False,False,False,False,False,False,False,False


# Cleaning data

In [139]:
df_train_clean = handle_missing_values(df_preprocessed_train, method='drop', numeric_strategy='median', categorical_strategy='mode')
# df_train_clean = handle_missing_values(df_preprocessed_train, method='fill', numeric_strategy='median', categorical_strategy='mode')
df_train_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 613 entries, 12 to 26999
Columns: 2563 entries, loan_amnt to last_credit_pull_d_Sep-15
dtypes: bool(2530), datetime64[ns](2), float64(18), int64(13)
memory usage: 1.6 MB


In [140]:
df_train_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 613 entries, 12 to 26999
Columns: 2563 entries, loan_amnt to last_credit_pull_d_Sep-15
dtypes: bool(2530), datetime64[ns](2), float64(18), int64(13)
memory usage: 1.6 MB


## Display correlation matrix



In [141]:
# [!Warning] Too slow
# Show correlation matrix
# plot_correlation_heatmap(df_encoded)

In [142]:
# [!Warning] Too slow
# Drop drop_highly_correlated_features
# new_df, dropped = drop_highly_correlated_features(df_encoded, threshold=0.75)
# new_df.head()

# Standardize numerical features

In [143]:
# Run standardSclar on clean and process dataset
# TODO: apply this separately to train and test data
df_train_clean_standardize = standardize_numerical_features(df_train_clean)

## Apply LASSO

In [144]:
model, selected_features, error = run_lasso_feature_selection(df_train_clean_standardize, target='loan_status')

DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float6

## Before Feature engineering

In [ ]:
# Feature selection
X = df_train_clean_standardize[selected_features]  # Features
# X = df_train_clean_standardize.drop(columns=['loan_status'])  # Features
y = (df_train_clean_standardize['loan_status'] > 0.5).astype(int)
X.info()

In [ ]:

# Handle class imbalance using SMOTE [analyse whether data is imblance first]
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [ ]:
# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "CatBoost": CatBoostClassifier(verbose=0),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

In [ ]:
# Train and evaluate models
results = {}
# plt.figure(figsize=(8, 6))

# Prepare a combined ROC plot
plt.figure(figsize=(10, 8))
colors = ['darkorange', 'blue', 'green', 'red', 'purple', 'brown']
color_idx = 0

for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)

    # Predictions on test data
    y_pred = model.predict(X_test)

    # Use predict_proba if available, otherwise decision_function
    if hasattr(model, "predict_proba"):
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        train_pred_proba = model.predict_proba(X_train)[:, 1]
    else:
        y_pred_proba = model.decision_function(X_test)
        train_pred_proba = model.decision_function(X_train)

    # Compute metrics and store them
    results[name] = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1-score": f1_score(y_test, y_pred),
        "AUC": roc_auc_score(y_test, y_pred_proba),
        "Train Accuracy": accuracy_score(y_train, model.predict(X_train)),
        "Test Accuracy": accuracy_score(y_test, y_pred),
        "Train AUC": roc_auc_score(y_train, train_pred_proba),
        "Test AUC": roc_auc_score(y_test, y_pred_proba)
    }

    # Print classification report for each model
    print(f"\n{name} Performance:")
    print(classification_report(y_test, y_pred))

    # Plot confusion matrix for each model
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['No Default', 'Default'],
                yticklabels=['No Default', 'Default'])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix - {name}")
    plt.show()

    # Compute ROC curve and add to the combined plot
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'{name} (AUC = {roc_auc:.4f})',
             color=colors[color_idx % len(colors)])
    color_idx += 1

# Finalize and show the combined ROC plot
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([-0.01, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison for All Models')
plt.legend(loc="lower right")
plt.show()

# Create and display a table comparing model results
results_df = pd.DataFrame(results).T  # Transpose to have models as rows
results_df = results_df.round(4)      # Round metrics for readability

print("\nComparison of Model Results:")
print(results_df)